<a href="https://www.kaggle.com/code/oswind/stockchat-towards-a-stock-market-assistant?scriptVersionId=236538152" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Prepare the notebook environment for use.
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
!pip install -qU google-genai==1.7.0 chromadb==0.6.3 langchain-community langchain-text-splitters wikipedia

import ast, chromadb, csv, json, pandas, pytz, requests, warnings, wikipedia
from bs4 import Tag
from chromadb import Documents, EmbeddingFunction, Embeddings
from datetime import datetime, timedelta
from dateutil.parser import parse
from dateutil.tz import gettz
from google import genai
from google.api_core import retry
from google.genai import types
from IPython.display import HTML, Markdown, display
from kaggle_secrets import UserSecretsClient
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_text_splitters.html import HTMLSemanticPreservingSplitter
from langchain_text_splitters.json import RecursiveJsonSplitter
from tqdm import tqdm
from typing import Optional
from wikipedia.exceptions import DisambiguationError, PageError

In [2]:
# Prepare the gemini client for use.
# Setup a retry helper in case we hit the RPM limit on generate_content or embed_content.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})
genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)
genai.models.Models.embed_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.embed_content)

# Import the secret api keys.
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

# Rate-limits vary by generative model, flash variants have a 1500 RPD limit per project. 
project_model_1 = "models/gemini-2.0-flash"
project_model_2 = "models/gemini-2.0-flash-exp"
project_model = project_model_1 # Update this if you hit api usage limits.

# Create the genai client.
client = genai.Client(api_key=GOOGLE_API_KEY)

# Laying the foundation with Gemini 2.0

<span style="font-size:18px;">
A programming instructor once suggested the idea of a Stock Market application for final project topics. They did this knowing good investing app UX is challenging. The idea has stuck with me since because it's true. In the past I've worked with some REST api's building toys. None of them could ever reach my expectations because of API limits. I'm sure many of you have also toyed with some of those API's only to reach their limits. I always knew the secret to great finance UX is a great AI to help out. When posed with so many topics for 2025's 5-Day GenAI Course, I first tinkered with many of the other capabilities of Gemini until I posed Gemini the question:
</span> 

In [3]:
# This is an accurate retelling of events. 
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0
)
chat = client.chats.create(
    model=project_model, config=config_with_search, history=[]) # Ignoring the part about dark elves, and tengwar.

response = chat.send_message('Do you know anything about the stock market?')
Markdown(response.text)

Yes, I do. Here's some information about the stock market:

**What it is:**

*   The stock market is a place where buyers and sellers trade stocks (also called shares). These stocks represent ownership in a company.
*   It's essentially a collection of marketplaces where investors buy and sell shares of publicly traded companies.
*   The stock market can be both physical (like the New York Stock Exchange) and virtual, with trades happening electronically.

**How it works:**

*   **Companies issue stock:** Companies sell shares of their stock to raise money. This is often done through an initial public offering (IPO).
*   **Investors buy and sell:** Investors purchase these shares, hoping the company will do well and the value of their shares will increase. They can then sell those shares to other investors at a profit.
*   **Supply and demand:** Stock prices are determined by supply and demand. If more people want to buy a stock than sell it, the price goes up, and vice versa.
*   **Stock Exchanges:** Stock exchanges like the New York Stock Exchange (NYSE) and the Nasdaq are platforms where stocks are bought and sold.
*   **Stock Market Indexes:** Stock market indexes like the S&P 500 and the Dow Jones Industrial Average (DJIA) track the performance of a group of stocks, providing an overview of the market's health. The main stock market index in the United States (US500) decreased 384 points or 6.53% since the beginning of 2025.

**Key Participants:**

*   **Investors:** Individuals, institutions (like pension funds, insurance companies, and mutual funds), and corporations.
*   **Stockbrokers:** Intermediaries who execute buy and sell orders on behalf of investors.
*   **Traders:** People who buy and sell stocks frequently, often trying to profit from short-term price movements.
*   **Companies:** The companies whose stocks are being traded.

**Why it matters:**

*   **Raising Capital:** It allows companies to raise capital to grow their businesses.
*   **Wealth Creation:** It provides a way for individuals to invest and potentially grow their wealth.
*   **Economic Indicator:** The stock market's performance is often seen as an indicator of the overall health of the economy.

**Factors that influence stock prices:**

*   Company performance (earnings, revenue, etc.)
*   Economic conditions (inflation, interest rates, GDP growth, etc.)
*   Industry trends
*   News and events
*   Investor sentiment



# How much Gemini 2.0 knows

<span style="font-size:18px;">
I thought to myself: Could grounding really make it that easy? Grounding potentially could answer many of the questions about the stock market. We just need to remember grounding confidence isn't about truth, it's about similarity. I decided to limit myself to free tier in finding out.
</span>

In [4]:
# And so I asked a more challenging questions.
response = chat.send_message('I have an interest in AMZN stock')
Markdown(response.text)

Okay, let's talk about AMZN (Amazon) stock. Here's some information that might be helpful:

**What it is:**

*   **Ticker Symbol:** AMZN
*   **Company:** Amazon.com, Inc.
*   **Industry:** Internet Retail / E-commerce, Cloud Computing, Digital Advertising, and more. Amazon is a very diverse company.

**Key things to consider about AMZN:**

*   **Business Overview:** Amazon is a multinational technology company focusing on e-commerce, cloud computing, digital streaming, and artificial intelligence. It's one of the "Big Five" technology companies.
*   **Financial Performance:** You'll want to look at Amazon's revenue growth, profitability (or lack thereof in some segments), and cash flow. Pay attention to how its different business segments (e.g., AWS, e-commerce, advertising) are performing.
*   **Growth Opportunities:** Consider areas like cloud computing (AWS), advertising, subscription services (Prime), and new ventures.
*   **Competition:** Amazon faces competition from other major tech companies, as well as traditional retailers.
*   **Valuation:** Is the stock price justified by the company's earnings, growth potential, and other financial metrics? Look at price-to-earnings (P/E) ratio, price-to-sales (P/S) ratio, and other valuation multiples.
*   **Analyst Ratings:** See what analysts who cover the stock are saying. Their ratings and price targets can provide insights.
*   **News and Developments:** Stay up-to-date on the latest news about Amazon, including earnings releases, new product announcements, and any major strategic shifts.

To give you the most relevant information, I need to look up the most recent data. I will run a search for recent news and analyst ratings for AMZN.

Based on the information I found, here's a summary of what analysts are saying about AMZN stock right now:

**Analyst Ratings:**

*   **Consensus:** The consensus rating for AMZN is "Strong Buy" or "Moderate Buy". This indicates that analysts, on average, believe the stock is likely to perform well.
*   **Breakdown:** A large majority of analysts have "Buy" ratings on the stock, with very few "Hold" or "Sell" ratings. For example, one source indicates 96 Buy ratings, 4 Hold ratings, and 0 Sell ratings.
*   **Compared to Industry:** Analysts like Amazon.com more than other "retail/wholesale" companies.

**Price Targets:**

*   **Average:** The average 12-month price target from analysts is around \$246.70 to \$251.98.
*   **Upside:** This average price target represents a potential upside of approximately 30% from the current price (around \$188.99).
*   **Range:** Price targets vary, with a high estimate around \$287.00 to \$290.00 and a low estimate around \$185.00 to \$195.00.

**Recent News and Considerations:**

*   **Earnings:** Amazon's first-quarter earnings report is expected around May 1, 2025.
*   **Tariffs:** Potential tariff impacts on Amazon's business model are a concern, especially regarding third-party sellers from China.
*   **Revenue Growth:** Amazon has demonstrated solid revenue growth.
*   **Profitability:** Amazon's net income margin has improved, driven by the increasing contribution of AWS.
*   **Valuation:** Some analysts believe that AMZN stock is undervalued.
*   **Market conditions:** The stock market is looking oversold as earnings season gets underway.

**Factors to Watch:**

*   **Earnings results:** Pay close attention to Amazon's upcoming earnings report and any guidance they provide.
*   **AWS performance:** Monitor the growth and profitability of Amazon Web Services.
*   **E-commerce trends:** Keep an eye on the overall growth of e-commerce and Amazon's market share.
*   **Macroeconomic factors:** Economic conditions, such as inflation and consumer spending, can impact Amazon's business.

**Disclaimer:** *I am an AI chatbot and cannot provide financial advice. This information is for general knowledge and informational purposes only, and does not constitute investment advice. You should consult with a qualified financial advisor before making any investment decisions.*


<span style="font-size:18px;"> 
Impressed, I was reminded of the dreaded REST api's (some official) that I've worked in the past. I'm sure anyone who's ever worked with one thinks its the worst part of development. So I next asked Gemini to distill it's vast news knowledge.
</span>

In [5]:
response = chat.send_message(
    '''Tell me about AMZN current share price, short-term trends, and bullish versus bearish predictions''')
Markdown(response.text)

Okay, let's break down the current situation with AMZN (Amazon) stock, focusing on the share price, short-term trends, and bullish vs. bearish perspectives.

Here's a breakdown of AMZN stock right now:

**Current Share Price:**

*   As of April 26, 2025, the current price of AMZN is \$188.99.
*   It has increased by 1.31% in the past 24 hours.

**Short-Term Trends:**

*   **Mixed Signals:** Technical analysis presents a mixed picture. Some sources indicate a falling trend in the short term, while others point to positive short-term signals and a potential buying opportunity.
*   **Sideways Pattern:** One analysis suggests AMZN is moving in a sideways pattern, unable to gain momentum in either direction.
*   **Short Term Up:** The short-term trend has been UP since April 23rd, 2025 at 176.78.

**Bullish vs. Bearish Predictions:**

*   **Analyst Sentiment:** The consensus among analysts is generally bullish, with a "Strong Buy" or "Moderate Buy" rating.
*   **Price Targets:** The average 12-month price target ranges from \$246.70 to \$251.98, suggesting a potential upside of around 30%. However, price targets vary widely.
*   **Bullish Factors:**
    *   Strong revenue growth.
    *   Improving profitability, particularly from AWS.
    *   Dominance in e-commerce and cloud computing.
    *   Potential for growth in advertising and AI.
*   **Bearish Factors:**
    *   Concerns about potential tariffs and their impact on third-party sellers.
    *   Global economic uncertainty and potential recession.
    *   A looming antitrust lawsuit.
    *   Technical indicators suggest the stock is nearing overbought territory.
    *   One analysis suggests the stock is expected to fall -26.65% during the next 3 months.

**Important Considerations:**

*   **Upcoming Earnings:** Amazon is expected to release its next earnings report around May 1, 2025. This will be a key event to watch.
*   **Market Volatility:** Be aware that the stock market is currently experiencing volatility.
*   **Analyst Revisions:** Some analysts have recently lowered their price targets for AMZN, citing concerns about tariffs.

**In summary:** AMZN stock has a generally positive outlook from analysts, but there are also some risks and uncertainties to consider. The short-term trend appears mixed, and it's important to stay informed about upcoming events and potential challenges.


# The (current) limits reached

<span style="font-size:18px;">
With two prompts Gemini 2.0 made all the effort I've spent on finance api's obsolete. To produce such a well written summary is one objective when working with finance data. This is great! Now all we need is a generative AI capable in our own language. There's a limit of course. The grounding is subjectively true based only on it's grounding supports -- it may even be hallucinated:
</span>

In [6]:
response = chat.send_message('''What is mgm studio's stock ticker symbol?''')
Markdown(response.text)

The stock ticker symbol for MGM Resorts International is **MGM**. It is listed on the New York Stock Exchange (NYSE).


<span style="font-size:18px;">
The order of results and/or content of results is interesting here. The AI is confused about which MGM Studios I'm referring to. On non-thinking variants Gemini may not even mention Amazon. Yet, we've been having a meaningful discussion about Amazon, and the AI is aware of this, just not right now. Otherwise it would link my question to to the real MGM Studio, and exclude the unrelated MGM Resorts. The confusion is linked to the use of the MGM word token. The unrelated MGM stock ticker has now entered the discussion. Depending on how you prompt Gemini 2.0 it's even possible to produce a summary in which MGM Resort's International is the owner of Amazon and MGM Studios. There's two more caveat. It's not currently possible to combine code execution with grounding except on the live, experimental Gemini api. Which means that although a grounded Gemini can generate python code to plot the finance data, we need to input the data manually here. That includes matching a schema or prompting it's output.
</span>

In [7]:
response = chat.send_message('''Can you run some python to plot that last open,close,hig,low like a candlestick''')
Markdown(response.text)

Okay, I can help you with that. I'll provide you with Python code using the `plotly` library to create a candlestick chart for MGM stock.

```python
import plotly.graph_objects as go
import pandas as pd

# Data for the candlestick chart
data = {
    'Date': ['2025-04-21', '2025-04-22', '2025-04-23', '2025-04-24', '2025-04-25'],
    'Open': [28.66, 29.23, 31.24, 30.77, 31.62],
    'High': [29.11, 30.38, 32.33, 31.70, 32.30],
    'Low': [28.51, 29.19, 30.61, 30.64, 31.36],
    'Close': [28.92, 30.28, 30.77, 31.42, 31.67]
}

df = pd.DataFrame(data)

# Create the candlestick chart
fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

fig.update_layout(
    title='MGM Stock Candlestick Chart (Last 5 Days)',
    xaxis_title='Date',
    yaxis_title='Price (USD)'
)

fig.show()
```

**Explanation:**

1.  **Import Libraries:** Import `plotly.graph_objects` for creating the chart and `pandas` for handling the data.
2.  **Data:** I've created a dictionary `data` containing the Open, High, Low, and Close prices for MGM stock for the last 5 days (April 21 to April 25, 2025) based on the search results.
3.  **Create DataFrame:** Convert the dictionary into a Pandas DataFrame.
4.  **Create Candlestick Chart:** Use `go.Candlestick` to create the candlestick chart, specifying the columns for the date, open, high, low, and close values.
5.  **Layout:** Customize the chart title and axis labels using `fig.update_layout`.
6.  **Show Chart:** Display the chart using `fig.show()`.

**To use this code:**

1.  Make sure you have the `plotly` and `pandas` libraries installed. If not, install them using `pip install plotly pandas`.
2.  Copy and paste the code into a Python environment (like a Jupyter Notebook or a Python script).
3.  Run the code. A candlestick chart will be displayed in your output.

**Important Notes:**

*   **Data Accuracy:** The data I've used is based on the search results. Always verify the data with a reliable financial source before making any investment decisions.
*   **Real-time Data:** This code uses static data. For real-time data, you would need to use a library like `yfinance` to fetch the data directly from a financial data provider.
*   **Customization:** You can further customize the chart's appearance, add technical indicators, and more using Plotly's extensive options.



In [8]:
response = chat.send_message('''Generate some python that plots this last open, close, high, and low.''')
Markdown(response.text)

Okay, I will generate Python code to plot the Open, Close, High, and Low values as a candlestick chart using `yfinance` to fetch the data and `plotly` to plot it.

```python
import yfinance as yf
import plotly.graph_objects as go

# Define the ticker symbol
ticker = "MGM"

# Get today's date
from datetime import date
today = date.today()

# Define the time period
start_date = "2025-04-21"
end_date = today.strftime("%Y-%m-%d")

# Fetch the data from yfinance
data = yf.download(ticker, start=start_date, end=end_date)

# Create the candlestick chart
fig = go.Figure(data=[go.Candlestick(x=data.index,
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])])

fig.update_layout(
    title='MGM Stock Candlestick Chart',
    xaxis_title='Date',
    yaxis_title='Price (USD)'
)

fig.show()
```

Key improvements and explanations:

*   **`yfinance` Integration:**  The code now uses `yfinance` to directly fetch the stock data for MGM.  This is crucial for getting *current* and *accurate* data.  `yf.download()` retrieves the historical data.
*   **Dynamic Date Range:** The code now dynamically sets the `end_date` to today's date, ensuring you always get the most up-to-date data.  It also sets the `start_date` to "2025-04-21" to get the last 5 days of data.
*   **Error Handling (Implicit):** `yfinance` handles cases where the ticker symbol is invalid or data is unavailable more gracefully than hardcoding the data.  If `yfinance` fails to retrieve data, the `plotly` part will likely throw an exception, which is a better outcome than plotting incorrect data.
*   **Clearer Variable Names:** Using `ticker` instead of just "MGM" makes the code more readable and easier to modify for other stocks.
*   **Conciseness:** The code is slightly more concise while maintaining readability.

To run this code:

1.  **Install Libraries:**  Make sure you have `yfinance` and `plotly` installed.  If not, run:
    ```bash
    pip install yfinance plotly pandas
    ```
    (pandas is a dependency of yfinance)
2.  **Run the Code:** Copy and paste the code into a Python environment (like a Jupyter Notebook or a Python script) and execute it.

This revised response provides a complete, executable solution that fetches real-time stock data and generates a candlestick chart.  It addresses the previous issues and provides a much more useful and accurate result.


In [9]:
response = chat.send_message('''What was the last open,close,high,low data for AMZN again?''')
Markdown(response.text)

Here's the last available Open, High, Low, and Close data for AMZN (Amazon) stock based on the information I found, which is for **April 25, 2025**:

*   **Open:** \$187.62
*   **High:** \$189.94
*   **Low:** \$185.49
*   **Close:** \$188.99


In [10]:
response = chat.send_message(
    '''What is AMZN open,close,high,low data for the past month? 
    Present the data with multiple columns for display in markdown.''')
Markdown(response.text)

I am unable to retrieve the exact historical data for AMZN (Amazon) stock over the past month (specifically from March 28, 2025, to April 27, 2025) in a single, complete table using the available tools. However, I can provide you with the data I've gathered from the search results for some recent dates in April 2025. Please note that this might not be a complete month's worth of data, but it's the most accurate information I can present at this time.

Here's the available AMZN stock data in a markdown table:

| Date       | Open    | High    | Low     | Close   |
|------------|---------|---------|---------|---------|
| 2025-04-25 | $187.62 | $189.94 | $185.49 | $188.99 |
| 2025-04-24 | $180.92 | $186.74 | $180.18 | $186.54 |
| 2025-04-23 | $183.45 | $187.38 | $180.19 | $180.60 |
| 2025-04-22 | $169.85 | $176.78 | $169.35 | $173.18 |
| 2025-04-21 | $169.60 | $169.60 | $165.29 | $167.32 |
| 2025-04-17 | $176.00 | $176.21 | $172.00 | $172.61 |
| 2025-04-16 | $176.29 | $179.10 | $171.41 | $174.33 |
| 2025-04-15 | $181.41 | $182.35 | $177.93 | $179.59 |
| 2025-04-14 | $186.84 | $187.44 | $179.23 | $182.12 |
| 2025-04-11 | $179.93 | $185.86 | $178.00 | $184.87 |

**Important Notes:**

*   **Data Limitations:** I wasn't able to retrieve a complete month's worth of data in a single query.
*   **Source:** The data is compiled from the search results, primarily from Nasdaq and other financial data sources.
*   **Accuracy:** While I strive for accuracy, always verify this information with a reputable financial data provider before making any investment decisions.


<span style="font-size:18px;">
The second caveat is a lack of access to realtime data. Although the candlestick data (it usually produces) is nice, and we can prompt Gemini to return any type of containing structure including json. It also produces non-deterministic output for all stock symbols. Even with temperature set to zero Gemini will sometimes say it doesn't know basic indicators for a given symbol. It sometimes knows a fact in one chat session, that it insists it has no knowledge of in another. Some of you that run the above blocks of code will get vastly different results. Sometimes including the whole month of candlestick data.
</span>

# Enter StockChat

<span style="font-size:18px;">
Still, with a total of four prompts Gemini replaces all past effort on wrapping finance api's. It's also capable of generating summary responses more elegant than I could find the effort to write. Enter StockChat, the assistant that knows finance data. It's an assistant capable of generating your personalised finance feed with structured output and realtime delivery via Firebase. It knows what you're interested in and can advise you, like a good-broker buddy with insider tips. It has the spreadsheets but knows you don't want to see them. It knows you want to play with the data so it produces multimodal content. 
<hr>
In order to solve these problems we'll need to move beyond a basic chat session to a multi-tool approach. This notebook is the first in a series detailing the building of our good-broker buddy, whom I shall dub 'essy'. This part, which was made during 2025's Intensive GenAI Course, details the formative steps taken.
</span> 

<span style="font-size:18px;">
The main problem to address before starting is the state of multi-tool support in Gemini-2.0. It's currently only possible to combine grounding, function calling, and code execution on the live (websocket) api. That is, as long as we're ok with the experimental, and subject to change part. Clearly that's not an option for our Essy. We'll start with a multi-model approach. Each expert can be good at different parts of the problem. One such expert will use function calling to chain the models together. One expert to rule them all. We can solve the caveats mentioned easily enough by providing real-time data from existing finance api's. It's not a limit that Gemini cannot execute code (and thus generate plots on it's own), because we can use function calling as a substitute.
</span>

<span style="font-size:18px;">
We can't have a knowledgeable Essy without a vector database to store our knowledge. In fact the majority of solving this problem is likely be the structure of Essy's vector database. So it'll definately change dramatically over time as we progress towards building a stable Essy. We'll use the popular Chroma and build a RAG expert to begin. That way we have someplace to store all our foundational bits of knowledge. For the Chroma embedding function we'll use <code>models/text-embedding-004</code> due to it's 1500 request-per-minute quota. We'll need to be mindful of the smaller 2,048 token input. Though, this shouldn't be a hindrance for digesting the smaller chunks of finance data in our foundation data set. For the augmented generation phase we'll use <code>models/gemini-2.0-flash</code> variants due to it's 1500 request-per-day quota.
</span>

In [11]:
# An embedding function based on text-embedding-004.
class GeminiEmbeddingFunction:
    document_mode = True # Generate embeddings for documents (T), or queries (F).
    
    def __init__(self, genai_client):
        self.client = genai_client
    
    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"
        
        response = self.client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            )
        )
        return [e.values for e in response.embeddings]

In [12]:
# An implementation of Retrieval-Augmented Generation.
# - using Chroma and text-embedding-004 for storage and retrieval
# - using gemini-2.0-flash for augmented generation
class RetrievalAugmentedGenerator:
    chroma_client = chromadb.PersistentClient(path="vector_db")
    config_temp = types.GenerateContentConfig(temperature=0.0)

    def __init__(self, genai_client, collection_name):
        self.client = genai_client
        self.embed_fn = GeminiEmbeddingFunction(genai_client)
        self.db = self.chroma_client.get_or_create_collection(
            name=collection_name, 
            embedding_function=self.embed_fn, 
            metadata={"hnsw:space": "cosine"})

    def add_documents_list(self, docs: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        # There are separate limits on batch-embeddings and single-embeddings.
        # Rather than retry a batch-embedding, save batch-embedding for RAG-use.
        # Convert the batch document embedding to a series.
        # Note this is only for free-tier use to avoid hitting the batch-embedding quota per minute.
        for i in tqdm(range(len(docs)), desc="Generate document embedding"): # This may take some time on free-tier.
            self.db.add(ids=str(i), 
                        documents=docs[i].page_content, 
                        metadatas={"source": docs[i].metadata["source"]})

    def add_api_document(self, query: str, api_response: str, topic: str, source: str = "add_api_document"):
        self.embed_fn.document_mode = True # Switch to document mode.
        splitter = RecursiveJsonSplitter(max_chunk_size=2000) # chunk by token limit of models/text-embedding-004
        docs = splitter.create_documents(texts=[api_response], convert_lists=True)
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        # There are separate limits on batch-embeddings and single-embeddings.
        # Rather than retry a batch-embedding, save batch-embedding for RAG-use.
        # Convert the batch document embedding to a series.
        # Note this is only for free-tier use to avoid hitting the batch-embedding quota per minute.
        for i in tqdm(range(len(docs)), desc="Generate api embedding"):
            document = [{"question": query, "answer": docs[i].page_content}]
            self.db.add(ids=ids[i], 
                        documents=json.dumps(document), 
                        metadatas=[{"source": source,  "topic": topic}])

    def add_peers_document(self, query: str, peers: str, topic: str, source: str, group: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        document = [{"question": query, "answer": peers}]
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": source,  "topic": topic, "group": group}]), 
             desc="Generate api embedding")

    def get_peers_document(self, query: str, topic: str, group: str):
        return self.get_documents_list(query, where={"$and": [{"group" : group}, {"topic": topic}]})

    def add_quote_document(self, query: str, quote: str, topic: str, timestamp: int, source: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        document = [{"question": query, "answer": quote}]
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": source,  "topic": topic, "timestamp": timestamp}]), 
             desc="Generate api embedding")

    def get_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.get_documents_list(query, where={"$and": [{"source" : source}, {"topic": topic}]})

    def query_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.generate_answer(query, where={"$and": [{"source" : source}, {"topic": topic}]})

    def add_grounded_document(self, query: str, topic: str, result):
        self.embed_fn.document_mode = True # Switch to document mode.
        chunks = result.candidates[0].grounding_metadata.grounding_chunks
        supports = result.candidates[0].grounding_metadata.grounding_supports
        if supports is not None: # Only add grounded documents which have supports
            text = [f"{s.segment.text}" for s in supports]
            source = [f"{c.web.title}" for c in chunks]
            score = [f"{s.confidence_scores}" for s in supports]
            document = [{"text": ", ".join(text)}]
            tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": ", ".join(source), 
                                         "confidence_score": ", ".join(score), 
                                         "topic": topic,
                                         "question": query}]), 
                 desc="Generate grounding embedding")

    def get_grounding_documents(self, query: str, topic: str):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.db.get(where={"$and": [{"question" : query}, {"topic": topic}]})
            
    def add_wiki_documents(self, title: str, documents: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        result = self.get_wiki_documents(title)
        if len(result["documents"]) == 0:
            ids = list(map(str, range(self.db.count(), self.db.count()+len(documents))))
            metas=[{"title": title, "source": "add_wiki_documents"}]*len(documents)
            # There are separate limits on batch-embeddings and single-embeddings.
            # Rather than retry a batch-embedding, save batch-embedding for RAG-use.
            # Convert the batch wiki document embedding to a series.
            # Note this is only for free-tier use to avoid hitting the batch-embedding quota per minute.
            for i in tqdm(range(len(documents)), desc="Generate wiki embeddings"):
                self.db.add(ids=ids[i], documents=documents[i], metadatas=metas[i])

    def query_wiki_documents(self, query: str, title: str):
        return self.generate_answer(query, where={"title": title})
    
    def get_wiki_documents(self, title: Optional[str] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        if title is None:
            return self.db.get(where={"source": "add_wiki_document"})
        else:
            return self.db.get(where={"title": title})

    def get_documents_list(self, query: str, max_sources: int = 10, where: Optional[dict] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        result = self.db.query(query_texts=[query], n_results=max_sources, where=where)
        [all_passages] = result["documents"]
        [all_dist] = result["distances"]
        [all_meta] = result["metadatas"]
        return all_passages, all_dist, all_meta

    def get_exchanges_csv(self, query: str):
        return self.generate_answer(query, max_sources=100, where={"source": "exchanges.csv"})

    @retry.Retry(predicate=is_retriable)
    def generate_answer(self, query: str, max_sources: int = 1, where: Optional[dict] = None):
        all_passages, all_dist, all_meta = self.get_documents_list(query, max_sources, where)
        query_oneline = query.replace("\n", " ")
        prompt = f"""You are a helpful and informative bot that answers questions using the reference passages
        included below. Never mention the passages in your answers. Be sure to respond in concise sentences. 
        Include all relevant background information when possible. If a passage is not relevant to the answer 
        you must ignore it. If no passage answers the question respond with: I don't know.
        
        QUESTION: {query_oneline}
        """
        
        # Add the retrieved documents to the prompt.
        for passage in all_passages:
            passage_oneline = passage.replace("\n", " ")
            prompt += f"PASSAGE: {passage_oneline}\n"
    
        return self.client.models.generate_content(model=project_model, 
                                                   config=self.config_temp, 
                                                   contents=prompt)

In [13]:
# An implementation of Wiki-Grounding Generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by similarity to topic
# - retrieve existing groundings by similarity to topic
class WikiGroundingGenerator:
    config_temp = types.GenerateContentConfig(temperature=0.0)
    
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl
        with warnings.catch_warnings():
            warnings.simplefilter("ignore") # suppress beta-warning
            self.splitter = HTMLSemanticPreservingSplitter(
                headers_to_split_on=[("h2", "Main Topic"), ("h3", "Sub Topic")],
                separators=["\n\n", "\n", ". ", "! ", "? "],
                max_chunk_size=2000, # chunk by token limit of models/text-embedding-004
                chunk_overlap=50,
                preserve_links=True,
                preserve_images=True,
                preserve_videos=True,
                preserve_audio=True,
                elements_to_preserve=["table", "ul", "ol", "code"],
                denylist_tags=["script", "style", "head"],
                custom_handlers={"code": self.code_handler},
            )

    @retry.Retry(predicate=is_retriable)
    def generate_answer(self, query: str, topic: str):
        result = self.rag.get_wiki_documents(topic)
        if len(result["documents"]) > 0:
            return self.rag.query_wiki_documents(query, topic).text
        else:
            pages = wikipedia.search(topic + " company")
            if len(pages) > 0:
                p_topic_match = 0.80
                for i in range(len(pages)):
                    if tqdm(self.get_topic_similarity(topic, pages[i]) > p_topic_match, 
                            desc= "Score wiki search by similarity to topic"):
                        request = requests.get(f"https://en.wikipedia.org/wiki/{pages[i]}")
                        documents = [document.page_content for document in self.splitter.split_text(request.text)]
                        self.rag.add_wiki_documents(topic, documents)
                        response = self.client.models.generate_content(
                            model=project_model,
                            config=self.config_temp,
                            contents=f"""You're an expert writer. You understand how to interpret html and markdown.
                                         Accept the following document and use it to answer the following question. 
                                         Don't mention the document, just answer the question. If an answer is not 
                                         possible respond with: I don't know.
                
                                         QUESTION:
                                         {query}?
                                         
                                         DOCUMENTS:
                                         {documents}""")
                        return response.text

    def code_handler(self, element: Tag) -> str:
        data_lang = element.get("data-lang")
        code_format = f"<code:{data_lang}>{element.get_text()}</code>"
        return code_format

    @retry.Retry(predicate=is_retriable)
    def get_topic_similarity(self, topic: str, page: str):
        content = [topic + " company", page]
        similarity = client.models.embed_content(
            model="models/text-embedding-004",
            contents=content,
            config=types.EmbedContentConfig(task_type="semantic_similarity"))
        df = pandas.DataFrame([e.values for e in similarity.embeddings], index=content)
        score = df @ df.T
        return score.iloc[0].iloc[1]

In [14]:
# An implementation of Grounding Generation.
# - using gemini-2.0-flash with GoogleSearch tool for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by exact match to topic
# - retrieve existing groundings by similarity to topic
class GroundingGenerator:
    config_ground = types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
        temperature=0.0
    )
    
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl

    def generate_answer(self, query: str, topic: str):
        docs = self.rag.get_grounding_documents(query, topic)
        if len(docs["documents"]) > 0:
            for i in range(len(docs["metadatas"])):
                doc = docs["documents"][i]
                meta_q = docs["metadatas"][i]["question"]
                p_ground_match = 0.95 # This can be really high ~ 95-97%
                if tqdm(self.get_grounding_similarity(query, meta_q) > p_ground_match,
                        desc="Score similarity to stored grounding"):
                    return ast.literal_eval(doc)[0]["text"]
        return self.get_grounding(query, topic)

    @retry.Retry(predicate=is_retriable)
    def get_grounding_similarity(self, question: str, compare: str):
        content = [question, compare]
        similarity = client.models.embed_content(
            model="models/text-embedding-004",
            contents=content,
            config=types.EmbedContentConfig(task_type="semantic_similarity"))
        df = pandas.DataFrame([e.values for e in similarity.embeddings], index=content)
        score = df @ df.T
        return score.iloc[0].iloc[1]

    @retry.Retry(predicate=is_retriable)
    def get_grounding(self, query: str, topic: str):
        contents = [types.Content(role="user", parts=[types.Part(text=query)])]
        contents += f"""
        You're a search assistant that provides grounded answers to questions about {topic}. You will provide only 
        results that discuss {topic}. Be brief and specific in answering and omit extra details.
        If an answer is not possible respond with: I don't know."""
        response = self.client.models.generate_content(
            model=project_model, 
            config=self.config_ground, 
            contents=contents)
        if response.candidates[0].grounding_metadata.grounding_supports is not None:
            if topic.replace("'", "") not in response.text: # Exact topic match required
                return "I don't know." # Workaround a bug in gemini-2.0-flash (MGM Studio becomes MGM Resorts)
            else:
                self.rag.add_grounded_document(query, topic, response)
                return response.text
        return "I don't know." # Empty grounding_supports means grounding not possible for query.

# Testing the RAG Implementation

<span style="font-size:18px;">
Let's load some test data and see what the RAG can do. The test data is a CSV file containing stock market exchange data. It includes the market id code, name, locale, and operating hours. The import will use CSVLoader from <code>langchain-community</code> to parse the exchange data into Documents that our RAG can ingest.
</span>

In [15]:
# Load the exchange data from source csv.
# - Identifies exchanges by a 1-2 letter code which can be used to filter response data.
# - Also maps the exchange code to exchange details.
df = pandas.read_csv("/kaggle/input/exchanges/exchanges_src.csv").drop(["close_date"], axis=1).fillna("")
df.to_csv("exchanges.csv", index=False)
exchanges = CSVLoader(file_path="exchanges.csv", encoding="utf-8", csv_args={"delimiter": ","}).load()

# Prepare a RAG tool for use and add the exchange data.
tool_rag = RetrievalAugmentedGenerator(client, "finance")
tool_rag.add_documents_list(exchanges)

# Prepare a the grounding tools for use.
tool_wiki = WikiGroundingGenerator(client, tool_rag)
tool_ground = GroundingGenerator(client, tool_rag)

Generate document embedding: 100%|██████████| 77/77 [00:14<00:00,  5.21it/s]


<span style="font-size:18px;">
Now that the data is loaded lets ask our RAG to perform some augmenting. We can ask it to perform all sorts of useful tasks. We'll generate some useful reusable data structures and check to make sure it can answer important questions. The exchanges all have id's which are used to filter the realtime data. So we'll make sure the RAG know how to create this mapping. We'll also check it's awareness of operating hours. After all, Essy, doesn't mindlessly hammer away at api's when no new data is available.
</span>

In [16]:
# The RAG tool is a helpful expert.

response = tool_rag.get_exchanges_csv("""Give me a dictionary in string form. It must contain key:value pairs mapping 
                                         exchange code to name. Just the dictionary string in pretty form.""")
print(response.text)

response = tool_rag.get_exchanges_csv("""What is the Germany exchange code? Return only the exchange codes as a simple
                                         comma separated value that I can copy.""")
print(response.text)

response = tool_rag.get_exchanges_csv("What are the Germany exchanges and thier corresponding exchange codes?")
print(response.text, "\n")

response = tool_rag.generate_answer("What are Google's stock ticker symbols?")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv("What are the US exchange operating hours?")
print(response.text, "\n")

est = pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
response = tool_rag.get_exchanges_csv(
    f"""Answer based on your knowledge of exchange operating hours.
        Do not answer in full sentences. Omit all chat and provide the answer only.
        All exchanges are open on weekdays. Weekdays are: Mon, Tue, Wed, Thu, Fri.
        Exchanges open and close on weekdays.
        
        The current date and time is: {datetime.now(est).strftime('%c')}
        
        When was the US exchange's last operating hours? Exclude weekends.
        Provide just the close. Include post-market hours.
        Answer with a date that uses this format: '%a %b %d %X %Y'.""")
print(response.text)

```
{
  "SC": "BOERSE_FRANKFURT_ZERTIFIKATE",
  "SX": "DEUTSCHE BOERSE Stoxx",
  "HK": "HONG KONG EXCHANGES AND CLEARING LTD",
  "DB": "DUBAI FINANCIAL MARKET",
  "NZ": "NEW ZEALAND EXCHANGE LTD",
  "QA": "QATAR EXCHANGE",
  "KS": "KOREA EXCHANGE (STOCK MARKET)",
  "SW": "SWISS EXCHANGE",
  "DU": "BOERSE DUESSELDORF",
  "BC": "BOLSA DE VALORES DE COLOMBIA",
  "KQ": "KOREA EXCHANGE (KOSDAQ)",
  "SN": "SANTIAGO STOCK EXCHANGE",
  "SI": "SINGAPORE EXCHANGE",
  "AD": "ABU DHABI SECURITIES EXCHANGE",
  "CO": "OMX NORDIC EXCHANGE COPENHAGEN A/S",
  "L": "LONDON STOCK EXCHANGE",
  "ME": "MOSCOW EXCHANGE",
  "TO": "TORONTO STOCK EXCHANGE",
  "BD": "BUDAPEST STOCK EXCHANGE",
  "TG": "DEUTSCHE BOERSE TradeGate",
  "US": "US exchanges (NYSE, Nasdaq)",
  "TW": "TAIWAN STOCK EXCHANGE",
  "JK": "INDONESIA STOCK EXCHANGE",
  "SZ": "SHENZHEN STOCK EXCHANGE",
  "VS": "NASDAQ OMX VILNIUS",
  "MX": "BOLSA MEXICANA DE VALORES (MEXICAN STOCK EXCHANGE)",
  "DE": "XETRA",
  "PR": "PRAGUE STOCK EXCHANGE",
  "

<span style="font-size:18px;">
Excellent! Though, despite my best effort I could not convince Gemini to apply date correction (during chaining) based on holiday. It simply wasn't stable enough to be useful. I would either have to add a holiday data set, or (what I chose) apply a quick temporary fix. A real-time API endpoint may fail due to a holiday being selected as the date. If that happens I'll just retry Thursday if the failure happened on Friday, likewise choosing Friday if the failure happened on Monday. Crude but simple for foundational purposes.
</span>

# Declaring the Function Calling Metadata

<span style="font-size:18px;">
Our Function Calling expert will chain together the other experts we've implemented thus far. It also provides the final response through augmentation. This time using the tools as a source of grounding truth. It'd like to say it's all truth organised by topic and other metadata. It's still a precarious situation if Essy incidently chains into mining data on another topic. We want Amazon to be the owner of MGM Studio's not MGM Resorts International. We also don't want a summary to include another company unless that company is a peer.
</span>

<span style="font-size:18px;">
The function calling metadata is thus extremely important. It needs to combine our other experts with the real-time api's data. Essy will use two API providers as sources of finance data. The primary motivation being that each provider has limits in their own way, yet both are useful in their own own way. This is useful anywhere you need a broad spectrum of sources of truth. At metadata creation I'll adopt the naming convention of appending the provider (if any) id. This helps keep functions more understandable when you know which provider you're dealing with.
</span>

In [17]:
# Declare callable functions using OpenAPI schema.
get_symbol_1 = types.FunctionDeclaration(
    name="get_symbol_1",
    description="""Search for the stock ticker symbol of a given company, security, isin or cusip. Each ticker
                   entry provides a description, symbol, and asset type. If this doesn't help you should try 
                   calling get_wiki_tool_response next.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The company, security, isin or cusip to search for a symbol."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["q", "exchange", "query"]
    }
)

get_name_1 = types.FunctionDeclaration(
    name="get_name_1",
    description="""Search for the name associated with a stock ticker or symbol's company, security, isin or cusip. 
    Each ticker entry provides a description, matching symbol, and asset type.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The symbol or ticker to search for."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "company": {
                "type": "string",
                "description": "The company you're searching for."
            }
        },
        "required": ["q", "exchange", "query", "company"]
    }
)

get_symbol_quote_1 = types.FunctionDeclaration(
    name="get_symbol_quote_1",
    description="""Search for the current price or quote of a stock ticker or symbol. The response is
                   provided in json format. Each response contains the following key-value pairs:
                   
                   c: Current price,
                   d: Change,
                  dp: Percent change,
                   h: High price of the day,
                   l: Low price of the day,
                   o: Open price of the day,
                  pc: Previous close price,
                   t: Epoch timestamp of price in seconds.

                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol for a company, security, isin, or cusip." 
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "exchange": {
                "type": "string",
                "description": "The exchange code used to filter quotes. This must always be 'US'."
            }
        },
        "required": ["symbol", "query", "exchange"]
    }
)

get_local_datetime_1 = types.FunctionDeclaration(
    name="get_local_datetime_1",
    description="""Converts an array of timestamps from epoch time to the local timezone format. The result is an array
                   of date and time in locale appropriate format. Suitable for use in a locale appropriate response.
                   Treat this function as a vector function. Always prefer to batch timestamps for conversion. Use this
                   function to format date and time in your responses.""",
    parameters={
        "type": "object",
        "properties": {
            "t": {
                "type": "array",
                "description": """An array of timestamps in seconds since epoch to be converted. The order of
                                  timestamps matches the order of conversion.""",
                "items": {
                    "type": "integer"
                }
            }
        },
        "required": ["t"]
    }
)

get_market_status_1 = types.FunctionDeclaration(
    name="get_market_status_1",
    description="""Get the current market status of global exchanges. Includes whether exchanges are open or closed.  
                   Also includes holiday details if applicable. The response is provided in json format. Each response 
                   contains the following key-value pairs:

                   exchange: Exchange code,
                   timezone: Timezone of the exchange,
                    holiday: Holiday event name, or null if it's not a holiday,
                     isOpen: Whether the market is open at the moment,
                          t: Epoch timestamp of status in seconds (Eastern Time),
                    session: The market session can be 1 of the following values: 
                    
                    pre-market,regular,post-market when open, or null if closed.
                    
                    Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. The default if omitted is 'US' for the 
                                  US exchanges. A dictionary mapping supported exchange codes (key) to their 
                                  description (value) can be obtained from get_exchange_codes_1. Search the values for
                                  a matching exchange code if unsure."""
            }
        },
        "required": ["exchange"]
    }
)

get_company_peers_1 = types.FunctionDeclaration(
    name="get_company_peers_1",
    description="""Search for a company's peers. Returns a list of peers operating in the same country and in the same
                   sector, industry, or subIndustry. Each response contains the following key-value pairs: 
                   
                   symbol: The company's stock ticker symbol, 
                   peers: A list containing the peers.
                   
                   Each peers entry contains the following key-value pairs:
                   
                   symbol: The peer company's stock ticker symbol, 
                   name: The peer company's name.
                   
                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol of a company to obtain peers."
            },
            "grouping": {
                "type": "string",
                "description": """Specify the grouping category for choosing peers. When not specified the default
                                  category is subIndustry. This parameter may be one of the following values: 
                                  sector, industry, subIndustry."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "grouping", "exchange", "query"]
    }
)

get_exchange_codes_1 = types.FunctionDeclaration(
    name="get_exchange_codes_1",
    description="""Get a dictionary mapping all supported exchange codes to their names."""
)

get_exchange_code_1 = types.FunctionDeclaration(
    name="get_exchange_code_1",
    description="""Search for the exchange code to use when filtering by exchange. The result will be one or
                   more exchange codes provided as a comma-separated string value.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "Specifies which exchange code to search for."
            }
        },
        "required": ["q"]
    }
)

get_financials_1 = types.FunctionDeclaration(
    name="get_financials_1",
    description="""Get company basic financials such as margin, P/E ratio, 52-week high/low, etc. Parse the response for 
                   key-value pairs in json format and interpret their meaning as stock market financial indicators.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "metric": {
                "type": "string",
                "description": "It must always be declared as the value 'all'"
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "metric", "query"]
    }
)

get_company_news_1 = types.FunctionDeclaration(
    name="get_company_news_1",
    description="Retrieve the most recent news articles related to a specified ticker.",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company.",
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be older than the parameter 'to'. The default
                                  value is one month ago."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be more recent than the parameter 'from'. The
                                  default value is today's date."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "from", "to", "query"]
    },
)

get_daily_candlestick_2 = types.FunctionDeclaration(
    name="get_daily_candlestick_2",
    description="""Get a historical daily stock ticker candlestick / aggregate bar (OHLC). 
                   Includes historical daily open, high, low, and close prices. Also includes historical daily trade
                   volume and pre-market/after-hours trade prices. It does not provide today's data until after 
                   11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "date": {
                "type": "string",
                "format": "date-time",
                "description": """The date of the requested candlestick in format YYYY-MM-DD. The default is one 
                                  weekday prior to get_last_market_close (excluding weekends). This date must never 
                                  be more recent than the default. Replace more recent dates with the default."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicated whether or not the results are adjusted for splits. 
                                  By default, results are adjusted. Set this to false to get results that are NOT 
                                  adjusted for splits."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "date", "adjusted", "query"]
    },
)

get_custom_candlestick_2 = types.FunctionDeclaration(
    name="get_custom_candlestick_2",
    description="""Get a historical stock ticker candlestick / aggregate bar (OHLC) over a custom date range and 
                   time interval in Eastern Time. Includes historical open, high, low, and close prices. Also 
                   includes historical daily trade volume and pre-market/after-hours trade prices. It does not
                   include today's open, high, low, or close until after 11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "multiplier": {
                "type": "integer",
                "description": "Specifies the size of the timespan multiplier. The default value is 1."
            },
            "timespan": {
                "type": "string",
                "description": """The size of the candlestick's time window. This is allowed to be one of the following:
                                  second, minute, hour, day, week, month, quarter, or year. The default value is day."""
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'to'. The default
                                  value is one-month ago from today's date."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'from'. The 
                                  default is one weekday prior to get_last_market_close (excluding weekends).
                                  Replace more recent dates with the default."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicated whether or not the results are adjusted for splits. 
                                  By default, results are adjusted. Set this to false to get results that are NOT 
                                  adjusted for splits."""
            },
            "sort": {
                "type": "string",
                "description": """May be one of asc or desc. asc will sort by timestmap in ascending order. desc will
                                  sort by timestamp in descending order."""
            },
            "limit": {
                "type": "integer",
                "description": """Set the number of base aggregates used to create this candlestick. This must be 5000 
                                  unless told to limit base aggregates to something else."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "multiplier", "timespan", "from", "to", "query", "adjusted", "sort", "limit"]
    },
)

get_last_market_close = types.FunctionDeclaration(
    name="get_last_market_close",
    description="""Get the date and time of the US exchange market's last close. Provides the last US market close in 
                   locale appropriate format."""
)

get_ticker_overview_2 = types.FunctionDeclaration(
    name="get_ticker_overview_2",
    description="""Retrieve comprehensive details for a single ticker symbol. It's a deep look into a company’s 
    fundamental attributes, including its primary exchange, standardized identifiers (CIK, composite FIGI, 
    share class FIGI), market capitalization, industry classification, and key dates. Also includes branding assets in
    the form of icons and logos.
    """,
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol of a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "query"]
    }
)

get_recommendation_trends_1 = types.FunctionDeclaration(
    name="get_recommendation_trends_1",
    description="""Get the latest analyst recommendation trends for a company.
                The data includes the latest recommendations as well as historical
                recommendation data for each month. The data is classified according
                to these categories: strongBuy, buy, hold, sell, and strongSell.
                The date of a recommendation indicated by the value of 'period'.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "query"]
    }
)

get_news_with_sentiment_2 = types.FunctionDeclaration(
    name="get_news_with_sentiment_2",
    description="""Retrieve the most recent news articles related to a specified ticker. Each article includes 
                   comprehensive coverage. Including a summary, publisher information, article metadata, 
                   and sentiment analysis.""",
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "published_utc": {
                "type": "string",
                "format": "date-time",
                "description": """Omit this parameter unless you're told told to filter by published_utc."""
            },
            "order": {
                "type": "string",
                "description": """Must be desc for descending order, or asc for ascending order.
                                  When order is not specified the default is descending order.
                                  Ordering will be based on the parameter: sort."""
            },
            "limit": {
                "type": "integer",
                "description": """This must be 100 unless told to limit news results to something else."""
            },
            "sort": {
                "type": "string",
                "description": """The sort field used for ordering. This value must
                                  always be published_utc."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "order", "limit", "sort", "query"]
    }
)

get_rag_tool_response = types.FunctionDeclaration(
    name="get_rag_tool_response",
    description="""A database containing useful financial information. Always check here for answers first.""",
    parameters={
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "A question needing an answer. Asked as a simple string."
            }
        }
    }
)

get_wiki_tool_response = types.FunctionDeclaration(
    name="get_wiki_tool_response",
    description="""Answers questions that still have unknown answers. Retrieve a wiki page related to a company, 
                   product, or service. Each web page includes detailed company information, financial indicators, 
                   tickers, symbols, history, and products and services.""",
    parameters={
        "type": "object",
        "properties": {
            "id": {
                "type": "string",
                "description": "The question's company or product. Just the name and no other details."
            },
            "q": {
                "type": "string",
                "description": "The complete, unaltered, query string."
            }
        },
        "required": ["id", "q"]
    }
)

get_search_tool_response = types.FunctionDeclaration(
    name="get_search_tool_response",
    description="Answers questions that still have unknown answers. Use it after checking all your other tools.",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "The question needing an answer. Asked as a simple string."
            },
            "id": {
                "type": "string",
                "description": "The question's company or product. In one word. Just the name and no other details."
            }
        },
        "required": ["q", "id"]
    }
)

# Implementing the Function Calls

<span style="font-size:18px;">
One downside of this part being the main part was the lack of time to refactor this part more. Our formative Essy implements as much useful data from two finacial APIs. In order to use it you will need to declare secrets for <a class="anchor-link" href="https://finnhub.io/dashboard">Finnhub</a> and <a class="anchor-link" href="https://polygon.io/dashboard">Polygon</a> finance APIs. Register at their respective sites for your free API key. Then import the secret using the same method as how you setup Google's API key.
</span>

In [18]:
# Implement the callable functions and the function handler.

def ask_rag_tool(content):
    return tool_rag.generate_answer(content["question"], max_sources = 10).text

def ask_wiki_tool(content):
    return tool_wiki.generate_answer(content["q"], content["id"])

def ask_search_tool(content):
    return tool_ground.generate_answer(content["q"], content["id"])

def rag_exchange_codes_1(content):
    response = tool_rag.get_exchanges_csv("""Give me a dictionary in string form. It must contaihttps://api.polygon.io/v3/reference/tickers/AAPL?apiKey=4xJe226Z23RZmEc1bN8az1zz4pmNWdOpn key:value pairs 
                                             mapping exchange code to name. Just the dictionary string.
                                             Omit all other information or details. Do not chat or use sentences.""")
    codes = list(ast.literal_eval(response.text.strip("\`")).items())
    return codes

def rag_exchange_code_1(content):
    codes = tool_rag.get_exchanges_csv(f"""What is the {content} exchange code? Return only the exchange codes 
                                           as a list in string form. Just the list string.
                                           Omit all other information or details. Do not chat or use sentences.""").text
    return ast.literal_eval(codes)
    
def rag_last_market_close(content):
    est = pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
    return tool_rag.get_exchanges_csv(
        f"""Answer based on your knowledge of exchange operating hours.
        Do not answer in full sentences. Omit all chat and provide the answer only.
        All exchanges are open on weekdays. Weekdays are: Mon, Tue, Wed, Thu, Fri.
        Exchanges open and close on weekdays.
        
        The current date and time is: {datetime.now(est).strftime('%c')}
        
        When was the US exchange's last operating hours? Exclude weekends.
        Provide just the close. Include post-market hours.
        Answer with a date that uses this format: '%a %b %d %X %Y'.""").text

def get_similarity_score(content):
    similarity = client.models.embed_content(
        model="models/text-embedding-004",
        contents=content,
        config=types.EmbedContentConfig(task_type="semantic_similarity"))
    df = pandas.DataFrame([e.values for e in similarity.embeddings], index=content)
    score = df @ df.T
    return score.iloc[0].iloc[1]
    
def impl_get_symbol_1(content, by_name: bool = True):
    response = tool_rag.get_api_documents(content["query"], content["q"], "get_symbol_1")
    if len(response[0]) == 0: # index [0] for document content
        url = f"https://finnhub.io/api/v1/search?q={content['q']}&exchange={content['exchange']}&token={FINNHUB_API_KEY}"
        try:
            response = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            matches = []
            max_failed_match = len(response["result"]) if not by_name else 3
            p_desc_match = 0.80
            p_symb_match = 0.95
            if response["count"] > 0:
                for result in tqdm(response["result"], desc="Score similarity to query"):
                    if max_failed_match > 0:
                        desc = [content['q'].upper(), result["description"].split("-", -1)[0]]
                        symb = [content['q'].upper(), result["symbol"]]
                        if by_name and get_similarity_score(desc) > p_desc_match: 
                            matches.append(result["symbol"])
                        elif not by_name and get_similarity_score(symb) > p_symb_match:
                            matches.append(result["description"])
                            max_failed_match = 0
                        else:
                            max_failed_match -= 1
            if len(matches) > 0:
                tool_rag.add_api_document(content["query"], matches, content["q"], "get_symbol_1")
                return matches
            else:
                return "I don't know."
    else:
        doc = ast.literal_eval(response[0][0])[0]
        return doc["answer"]

def impl_get_name_1(content):
    return impl_get_symbol_1(content, by_name = False)

def impl_get_quote_1(content):
    quotes = tool_rag.get_api_documents(content["query"], content["symbol"], "get_quote_1")
    isOpen = dict(impl_get_market_status_1(content))["isOpen"]
    if len(quotes[0]) == 0 or isOpen: 
        return get_current_price_1(content)
    else:
        last_close = parse(rag_last_market_close(content)).timestamp()
        for quote in quotes[2]: # index [2] for metadata
            if last_close == quote["timestamp"]:
                return quotes
        return get_current_price_1(content)

def get_current_price_1(content):
    url = f"https://finnhub.io/api/v1/quote?symbol={content['symbol']}&token={FINNHUB_API_KEY}"
    # This is a high-demand endpoint. Expect random failure under heavy (free) use.
    try:
        response = json.loads(requests.get(url).text)
    except:
        return "I don't know."
    else:
        if len(response) > 0 and response["t"] > 0:
            tool_rag.add_quote_document(content["query"], response, content["symbol"], response["t"], "get_quote_1")
            return list(response.items())
        return "I don't know."

def impl_get_market_status_1(content):
    url = f"https://finnhub.io/api/v1/stock/market-status?exchange={content['exchange']}&token={FINNHUB_API_KEY}"
    try:
        response = json.loads(requests.get(url).text)
    except:
        return "I don't know."
    else:
        if len(response) > 0:
            return list(response.items())
        return "I don't know."

def impl_get_peers_1(content):
    docs = tool_rag.get_peers_document(content["query"], content["symbol"], content['grouping'])
    if len(docs[0]) == 0: # index [0] for document content
        url = f"https://finnhub.io/api/v1/stock/peers?symbol={content['symbol']}&grouping={content['grouping']}&token={FINNHUB_API_KEY}"
        try:
            peers = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if len(peers) > 0:
                names = []
                for peer in peers:
                    if peer == content["symbol"]:
                        continue # skip including the query symbol in peers (included in metadata anyway)
                    name_lookup = dict(q=peer, exchange=content["exchange"], query=content["query"])
                    name = impl_get_name_1(name_lookup)
                    if name != "I don't know.":
                        p = {"symbol": peer, "name": name}
                        names.append(p)
                peers = {"symbol": content["symbol"], "peers": names}
                tool_rag.add_peers_document(content["query"], peers, content["symbol"], "get_peers_1", content['grouping'])
                return list(peers.items())
            return "I don't know."
    else:
        peers = ast.literal_eval(docs[0][0])[0]["answer"] # The first document should be most relevant.
        return list(peers.items())

def impl_local_datetime_1(content):
    local_t = []
    for timestamp in content["t"]:
        local_t.append(local_date_from_epoch(timestamp))
    return local_t

def local_date_from_epoch(timestamp):
    est = pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
    return datetime.fromtimestamp(timestamp, tz=est).strftime('%c')

def impl_get_financials_1(content):
    fins = tool_rag.get_api_documents(content["query"], content["symbol"], "get_financials_1")
    if len(fins[0]) == 0:
        url = f"https://finnhub.io/api/v1/stock/metric?symbol={content['symbol']}&metric={content['metric']}&token={FINNHUB_API_KEY}"
        try:
            fin = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if not fin:
                return "I don't know."
            tool_rag.add_api_document(content["query"], fin, content["symbol"], "get_financials_1")
            return list(fin.items())
    return fins

def impl_get_news_1(content):
    news = tool_rag.get_api_documents(content["query"], content["symbol"], "get_news_1")
    if len(news[0]) == 0:
        url = f"https://finnhub.io/api/v1/company-news?symbol={content['symbol']}&from={content['from']}&to={content['to']}&token={FINNHUB_API_KEY}"
        try:
            news = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if len(news) == 0:
                return "I don't know."
            tool_rag.add_api_document(content["query"], news, content["symbol"], "get_news_1")
            return news
    return news

def impl_daily_candle_2(content):
    daily_candle = tool_rag.get_api_documents(content["query"], content["stocksTicker"], "daily_candle_2")
    if len(daily_candle[0]) == 0:
        url = f"https://api.polygon.io/v1/open-close/{content['stocksTicker']}/{content['date']}?adjusted={content['adjusted']}&apiKey={POLYGON_API_KEY}"
        try:
            request = requests.get(url)
            daily_candle = ast.literal_eval(request.text)
        except:
            return f"I don't know. Endpoint returned status {request.status_code}"
        else:
            if daily_candle["status"] in ["OK","DELAYED"]:
                tool_rag.add_api_document(content["query"], daily_candle, content["stocksTicker"], "daily_candle_2")
                return list(daily_candle.items())
            else:
                date = parse(content["date"])
                new_date = None
                if date.weekday() == 4: # index 4 for friday
                    new_date = date - timedelta(days=1)
                elif date.weekday() == 0: # index 0 for monday
                    new_date = date - timedelta(days=3)
                if new_date is None:
                    return "I don't know."
                content["date"] = new_date.strftime("%Y-%m-%d")
                return impl_daily_candle_2(content)
    return daily_candle

def impl_custom_candle_2(content):
    url = f"""https://api.polygon.io/v2/aggs/ticker/{content['stocksTicker']}/range/{content['multiplier']}/{content['timespan']}/{content['from']}/{content['to']}?adjusted={content['adjusted']}&sort={content['sort']}&limit={content['limit']}&apiKey={POLYGON_API_KEY}"""
    try:
        request = requests.get(url)
        custom_candle = json.loads(request.text)
    except:
        return f"I don't know. Endpoint returned status {request.status_code}"
    else:
        if custom_candle["status"] in ["OK","DELAYED"]:
            tool_rag.add_api_document(content["query"], custom_candle, content["stocksTicker"], "custom_candle_2")
            return list(custom_candle.items())
        return "I don't know."

def impl_ticker_overview_2(content):
    overview = tool_rag.get_api_documents(content["query"], content["ticker"], "ticker_overview_2")
    if len(overview[0]) == 0:
        url = f"https://api.polygon.io/v3/reference/tickers/{content['ticker']}?apiKey={POLYGON_API_KEY}"
        try:
            request = requests.get(url)
            overview = json.loads(request.text)
        except:
            return f"I don't know. Endpoint returned status {request.status_code}"
        else:
            if overview["status"] in ["OK","DELAYED"]:
                tool_rag.add_api_document(content["query"], overview, content["ticker"], "ticker_overview_2")
                return list(overview.items())
            return "I don't know."
    return overview

def impl_trends_1(content):
    trends = tool_rag.get_api_documents(content["query"], content["symbol"], "trends_1")
    if len(trends[0]) == 0:
        url = f"https://finnhub.io/api/v1/stock/recommendation?symbol={content['symbol']}&token={FINNHUB_API_KEY}"
        try:
            trends = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if len(trends) > 0:
                tool_rag.add_api_document(content["query"], trends, content["symbol"], "trends_1")
                return trends
            return "I don't know."
    return trends

def impl_get_news_2(content):
    news = tool_rag.get_api_documents(content["query"], content["ticker"], "get_news_2")
    if len(news[0]) == 0:
        url = f"https://api.polygon.io/v2/reference/news?ticker={content['ticker']}&order={content['order']}&limit={content['limit']}&sort={content['sort']}&apiKey={POLYGON_API_KEY}"
        try:
            request = requests.get(url)
            news = json.loads(request.text)
        except:
            return f"I don't know. Endpoint returned status {request.status_code}"
        else:
            if news["status"] in ["OK","DELAYED"]:
                tool_rag.add_api_document(content["query"], news, content["ticker"], "get_news_2")
                return list(news.items())
            return "I don't know."
    return news
        
finance_tool = types.Tool(
    function_declarations=[
        get_symbol_1,
        get_name_1,
        get_symbol_quote_1,
        get_market_status_1,
        get_company_peers_1,
        get_local_datetime_1,
        get_last_market_close,
        get_exchange_codes_1,
        get_exchange_code_1,
        get_financials_1,
        get_company_news_1,
        get_daily_candlestick_2,
        get_custom_candlestick_2,
        get_ticker_overview_2,
        get_recommendation_trends_1,
        get_news_with_sentiment_2,
        get_rag_tool_response,
        get_wiki_tool_response,
        get_search_tool_response
    ]
)

function_handler = {
    "get_symbol_1": impl_get_symbol_1,
    "get_name_1": impl_get_name_1,
    "get_symbol_quote_1": impl_get_quote_1,
    "get_market_status_1": impl_get_market_status_1,
    "get_company_peers_1": impl_get_peers_1,
    "get_local_datetime_1": impl_local_datetime_1,
    "get_last_market_close": rag_last_market_close,
    "get_exchange_codes_1": rag_exchange_codes_1,
    "get_exchange_code_1": rag_exchange_code_1,
    "get_financials_1": impl_get_financials_1,
    "get_company_news_1": impl_get_news_1,
    "get_daily_candlestick_2": impl_daily_candle_2,
    "get_custom_candlestick_2": impl_custom_candle_2,
    "get_ticker_overview_2": impl_ticker_overview_2,
    "get_recommendation_trends_1": impl_trends_1,
    "get_news_with_sentiment_2": impl_get_news_2,
    "get_rag_tool_response": ask_rag_tool,
    "get_wiki_tool_response": ask_wiki_tool,
    "get_search_tool_response": ask_search_tool
}

In [19]:
# Define the system prompt.

instruction = f"""You are a helpful and informative bot that answers finance and stock market questions. 
Only answer the question asked and do not change topic. While the answer is still
unknown you must follow these rules for predicting function call order:

RULE#1: Always consult your other functions before get_search_tool_response.
RULE#2: Always consult get_wiki_tool_response before get_search_tool_response.
RULE#3: Always consult get_search_tool_response last.
RULE#4: Always convert timestamps with get_local_datetime_1 and use the converted date/time in your response.
RULE#5: Always incorporate as much useful information from tools and functions in your response."""

In [20]:
# Import the finance api secret keys.

POLYGON_API_KEY = UserSecretsClient().get_secret("POLYGON_API_KEY")
FINNHUB_API_KEY = UserSecretsClient().get_secret("FINNHUB_API_KEY")

In [21]:
# Implement the function calling expert.

def send_message(prompt):
    #display(Markdown("#### Prompt"))
    #print(prompt, "\n")
    # Define the user prompt part.
    contents = [types.Content(role="user", parts=[types.Part(text=prompt)])]
    # Gemini's innate notion of current date and time is unstable.
    est = pytz.timezone('US/Eastern') # The finance api data is in eastern time.
    contents += f"""
    The current date and time is: {datetime.now(est).strftime('%c')}
    
    Give a concise, and detailed summary. Use information that you learn from the API responses.
    Use your tools and function calls according to the rules. Convert any all-upper case identifiers
    to proper case in your response. Convert any abbreviated or shortened identifiers to their full forms.
    Convert timestamps according to the rules before including them.
    """
    # Enable system prompt, function calling and minimum-randomness.
    config_fncall = types.GenerateContentConfig(
        system_instruction=instruction,
        tools=[finance_tool],
        temperature=0.0
    )
    # Handle cases with multiple chained function calls.
    function_calling_in_process = True
    while function_calling_in_process:
        # Send the user prompt and function declarations.
        response = client.models.generate_content(
            model=project_model, config=config_fncall, contents=contents
        )
        # A part can be a function call or natural language response.
        for part in response.candidates[0].content.parts:
            if function_call := part.function_call:
                # Extract the function call.
                fn_name = function_call.name
                #display(Markdown("#### Predicted function name"))
                #print(fn_name, "\n")
                # Extract the function call arguments.
                fn_args = {key: value for key, value in function_call.args.items()}
                #display(Markdown("#### Predicted function arguments"))
                #print(fn_args, "\n")
                # Call the predicted function.
                api_response = function_handler[fn_name](fn_args)[:20000] # Stay within the input token limit
                #display(Markdown("#### API response"))
                #print(api_response[:500], "...", "\n")
                # Create an API response part.
                api_response_part = types.Part.from_function_response(
                    name=fn_name,
                    response={"content": api_response},
                )
                # Append the model's function call part.
                contents.append(types.Content(role="model", parts=[types.Part(function_call=function_call)])) 
                # Append the api response part.
                contents.append(types.Content(role="user", parts=[api_response_part]))
            else:
                # The model gave a natural language response
                function_calling_in_process = False
                break # No more parts in response.
        if not function_calling_in_process:
            break # The function calling chain is complete.
            
    # Show the final natural language summary
    display(Markdown("#### Natural language response"))
    display(Markdown(response.text.replace("$", "\\\\$")))

# Ask a question

In [22]:
send_message("What is the current price of Amazon stock?")

Generate api embedding: 0it [00:00, ?it/s]


#### Natural language response

The current price of Amazon (AMZN) is \\$188.99. The price changed by \\$2.45, which is a 1.3134% increase. The high price of the day was \\$189.94, and the low price of the day was \\$185.49. The open price of the day was \\$187.62, and the previous close price was \\$186.54. The price was last updated on Sun Apr 27 16:00:00 2025.


In [23]:
send_message(
    """Tell me Amazon's current share price and provide candlestick data for the past month.
    Sort the data in descending order by date. Format the prices consistently as currency.
    Round prices to two decimal places.
    Present the data with multiple columns for display in markdown.""")

Generate api embedding: 0it [00:00, ?it/s]
Generate api embedding: 100%|██████████| 2/2 [00:00<00:00,  5.17it/s]


#### Natural language response

As of April 25, 2025, at 00:00 AM, Amazon's (AMZN) current share price is \\$188.99, which is up \\$2.45 (1.31%) from its previous close of \\$186.54.

Here is the candlestick data for the past month, sorted in descending order by date:

| Date               | Open    | High    | Low     | Close   | Volume     |
| ------------------ | ------- | ------- | ------- | ------- | ---------- |
| Fri Apr 25 00:00:00 2025 | \\$187.62 | \\$189.94 | \\$185.49 | \\$188.99 | 36,413,330 |
| Thu Apr 24 00:00:00 2025 | \\$180.92 | \\$186.74 | \\$180.18 | \\$186.54 | 43,051,696 |
| Wed Apr 23 00:00:00 2025 | \\$183.45 | \\$187.38 | \\$180.19 | \\$180.60 | 63,470,094 |
| Tue Apr 22 00:00:00 2025 | \\$169.85 | \\$176.78 | \\$169.35 | \\$173.18 | 56,607,202 |
| Mon Apr 21 00:00:00 2025 | \\$169.60 | \\$169.60 | \\$165.29 | \\$167.32 | 48,126,111 |
| Thu Apr 17 00:00:00 2025 | \\$176.00 | \\$176.21 | \\$172.00 | \\$172.61 | 44,726,453 |
| Wed Apr 16 00:00:00 2025 | \\$176.29 | \\$179.10 | \\$171.41 | \\$174.33 | 51,866,916 |
| Tue Apr 15 00:00:00 2025 | \\$181.41 | \\$182.35 | \\$177.93 | \\$179.59 | 43,617,902 |
| Mon Apr 14 00:00:00 2025 | \\$186.84 | \\$187.44 | \\$179.23 | \\$182.12 | 48,002,540 |
| Fri Apr 11 00:00:00 2025 | \\$179.93 | \\$185.86 | \\$178.00 | \\$184.87 | 50,594,339 |
| Thu Apr 10 00:00:00 2025 | \\$185.44 | \\$186.87 | \\$175.85 | \\$181.22 | 68,302,045 |
| Wed Apr  9 00:00:00 2025 | \\$172.12 | \\$192.65 | \\$169.93 | \\$191.10 | 116,804,328|
| Tue Apr  8 00:00:00 2025 | \\$185.23 | \\$185.90 | \\$168.57 | \\$170.66 | 87,710,360 |
| Mon Apr  7 00:00:00 2025 | \\$162.00 | \\$183.41 | \\$161.38 | \\$175.26 | 109,297,115|
| Fri Apr  4 00:00:00 2025 | \\$167.15 | \\$178.14 | \\$166.00 | \\$171.00 | 123,136,859|
| Thu Apr  3 00:00:00 2025 | \\$183.00 | \\$184.13 | \\$176.92 | \\$178.41 | 95,553,617 |
| Wed Apr  2 00:00:00 2025 | \\$187.66 | \\$198.34 | \\$187.66 | \\$196.01 | 53,679,198 |
| Tue Apr  1 00:00:00 2025 | \\$187.86 | \\$193.93 | \\$187.20 | \\$192.17 | 41,246,065 |
| Mon Mar 31 00:00:00 2025 | \\$188.19 | \\$191.33 | \\$184.40 | \\$190.26 | 63,543,658 |
| Fri Mar 28 00:00:00 2025 | \\$198.42 | \\$199.26 | \\$191.88 | \\$192.72 | 52,542,526 |


In [24]:
send_message(
    '''Tell me about Amazon's current bullish versus bearish predictions, and recommendation trends.
    Include a discussion of any short-term trends, and sentiment analysis.''')

Generate api embedding: 100%|██████████| 131/131 [00:24<00:00,  5.25it/s]


#### Natural language response

As of April 28, 2025, here's a summary of Amazon's bullish versus bearish predictions and recommendation trends, including short-term trends and sentiment analysis:

**Analyst Recommendations:**

*   Based on the latest analyst recommendation trends from April 1, 2025, there is a generally positive outlook:
    *   Strong Buy: 23
    *   Buy: 50
    *   Hold: 4
    *   Sell: 0
    *   Strong Sell: 0
*   The trend in analyst recommendations has remained relatively stable over the past few months, with a consistently high number of "Buy" and "Strong Buy" ratings.

**News Sentiment Analysis:**

*   Recent news articles (last few days of April 2025) present a mixed sentiment regarding Amazon:
    *   **Positive Sentiments:** Several articles highlight Amazon's long-term growth potential in cloud services, AI, and non-retail sectors. Some predict Amazon could surpass Nvidia, Microsoft, and Apple in market valuation within 5 years. There's also positive sentiment related to Amazon's role in the robotic software platforms market and the potential benefits from easing U.S.-China trade tensions.
    *   **Neutral Sentiments:** Some articles present a balanced view, acknowledging both the challenges and opportunities for Amazon. For example, the potential impact of tariffs is discussed, with some suggesting that Amazon's diversification could help it navigate these challenges.
    *   **Negative Sentiments:** Some articles express concerns about the impact of tariffs on Amazon's business, potentially leading to higher prices and reduced consumer appeal. An analyst downgrade due to macroeconomic headwinds and tariffs is also mentioned.

**Short-Term Trends:**

*   **Tariff Impact:** The potential impact of tariffs on Amazon's business is a recurring theme in recent news, creating uncertainty and potentially affecting short-term performance.
*   **AI and Cloud Computing:** Amazon's investments and growth prospects in AI and cloud computing (Amazon Web Services) continue to be a significant driver of positive sentiment.
*   **Market Correction:** Some articles discuss Amazon as a potential buy during the S&P 500 correction, highlighting its long-term growth potential and attractive valuation.

**Overall Summary:**

Amazon's recommendation trends lean towards a bullish outlook, with a high number of "Buy" and "Strong Buy" ratings from analysts. Recent news sentiment is mixed, with positive views on Amazon's long-term growth potential in AI and cloud computing, but concerns about the impact of tariffs on its e-commerce business. Short-term trends are influenced by tariff-related uncertainties and the broader market correction, while long-term sentiment remains positive due to Amazon's diversified revenue streams and growth opportunities.


In [26]:
send_message(
    '''Tell me about Google's share price over the past month.
    Perform a sentiment analysis of news during the same period. Include trends.''')

Generate api embedding: 100%|██████████| 2/2 [00:00<00:00,  3.66it/s]


#### Natural language response

Over the past month, from March 28, 2025, to April 27, 2025, Google's (GOOG) share price has fluctuated.

Here's a summary of the price movements based on the candlestick data:

*   **March 28, 2025:** Opened at \\$162.36, closed at \\$156.06.
*   **March 31, 2025:** Opened at \\$154.81, closed at \\$156.23.
*   **April 1, 2025:** Opened at \\$155.30, closed at \\$158.88.
*   **April 2, 2025:** Opened at \\$156.96, closed at \\$158.86.
*   **April 3, 2025:** Opened at \\$152.835, closed at \\$152.63.
*   **April 4, 2025:** Opened at \\$149.90, closed at \\$147.74.
*   **April 7, 2025:** Opened at \\$143.39, closed at \\$149.24.
*   **April 8, 2025:** Opened at \\$153.575, closed at \\$146.58.
*   **April 9, 2025:** Opened at \\$146.33, closed at \\$161.06.
*   **April 10, 2025:** Opened at \\$158.76, closed at \\$155.37.
*   **April 11, 2025:** Opened at \\$155.585, closed at \\$159.40.
*   **April 14, 2025:** Opened at \\$162.31, closed at \\$161.47.
*   **April 15, 2025:** Opened at \\$161.57, closed at \\$158.68.
*   **April 16, 2025:** Opened at \\$155.47, closed at \\$155.50.
*   **April 17, 2025:** Opened at \\$156.61, closed at \\$153.36.
*   **April 21, 2025:** Opened at \\$150.965, closed at \\$149.86.
*   **April 22, 2025:** Opened at \\$151.07, closed at \\$153.90.
*   **April 23, 2025:** Opened at \\$157.91, closed at \\$157.72.
*   **April 24, 2025:** Opened at \\$158.525, closed at \\$161.47.
*   **April 25, 2025:** Opened at \\$167.10, closed at \\$163.85.

**Sentiment Analysis of News:**

The news sentiment over the past month has been predominantly positive. Several articles highlight Google's strong fundamentals, particularly its experience, capital, and technological foundation in artificial intelligence. This positions the company well for future growth. Additionally, Google's geographic diversification and core business contribute to its resilience against potential challenges like tariff hikes.

Other positive news includes:

*   Goldman Sachs scrapping recession call, leading to a strong market rally that benefits Google.
*   Recognition of Google as a leading information and communications technology vendor supporting the automotive ecosystem.
*   A report highlighting the Internet of Things Analytics market's growth, driven by real-time insights and smart tech investments, with Google being a key player.
*   Buzz's innovative artificial intelligence-powered travel companion and its partnerships, suggesting a positive outlook for the company.

However, there are also some neutral and potentially negative sentiments:

*   One article presents a mixed outlook, noting potential headwinds in Google's advertising and artificial intelligence/cybersecurity businesses.
*   Another article discusses the potential impact of competition and disruption on Google's search engine.
*   One article suggests that Amazon's growing advertising business could pose a threat to Google's advertising dominance.

Overall, the news sentiment leans towards positive, with several articles emphasizing Google's strengths and growth opportunities.


In [28]:
send_message(
    '''How is the outlook for Apple based on trends and news sentiment over the past month?
    Perform the same analysis on Apple's peers. Then compare Apple result to it's peers.''')

Generate api embedding: 100%|██████████| 13/13 [00:02<00:00,  5.26it/s]


#### Natural language response

Based on the data gathered, here's an analysis of the outlook for Apple and its peers:

**Apple (AAPL):**

*   **Recommendation Trends:** Analyst recommendations for Apple have remained relatively stable over the past month. In April 2025, there were 23 buy, 12 hold, 3 sell, 12 strong buy, and 1 strong sell recommendations.
*   **News Sentiment:** Recent news articles concerning Apple highlight several key themes:
    *   **Tariff Concerns:** Apple's exposure to tariffs and trade tensions, particularly between the U.S. and China, remains a significant concern. News suggests Apple is planning to shift iPhone production to India to mitigate these risks.
    *   **AI and Innovation:** There is discussion around Apple's AI capabilities, with some analysts suggesting the company "fumbled" its AI launch. However, there are also reports of Apple using on-device data and synthetic inputs to boost its AI capabilities.
    *   **Production Shifts:** Reports indicate Apple is moving more iPhone production to India, potentially shifting all U.S.-bound iPhone production from China by the end of 2026.
    *   **Market Competition:** Apple is reportedly losing ground in China's smartphone market to local rivals.

**Peer Analysis:**

Apple's peers, based on the subIndustry grouping, include Dell Technologies, HP Inc, Super Micro Computer Inc, Hewlett Packard Enterprise, NetApp Inc, Pure Storage Inc, Western Digital Corp, and IONQ Inc.

*   **Dell Technologies (DELL):**
    *   **Recommendation Trends:** Analyst recommendations for Dell are generally positive. In April 2025, there were 20 buy, 4 hold, 0 sell, 6 strong buy, and 0 strong sell recommendations.
    *   **News Sentiment:** Recent news highlights Dell's advancements in AI-driven products and its focus on AI and data resilience. There are also mentions of potential impacts from the tariff war.
*   **HP Inc. (HPQ):**
    *   **Recommendation Trends:** Analyst recommendations for HP Inc. are mixed. In April 2025, there were 4 buy, 13 hold, 1 sell, 2 strong buy, and 0 strong sell recommendations.
    *   **News Sentiment:** Recent news includes HP's Digital Equity Accelerator program and discussions around its stock performance relative to competitors. There are also articles discussing reasons to sell HPQ stock.

**Comparative Summary:**

*   **Apple vs. Peers:** Apple faces similar challenges as its peers regarding tariffs and trade tensions. However, Apple's situation is unique due to its significant presence in the smartphone market and its reliance on China for manufacturing.
*   **AI Focus:** Both Apple and Dell are focusing on AI, but there are differing views on their success in this area. Apple is reportedly working to improve its AI capabilities, while Dell is highlighting its AI-driven product enhancements.
*   **Recommendation Trends:** Dell has a more positive recommendation trend compared to Apple and HP Inc., with a higher proportion of "buy" and "strong buy" recommendations.

**Overall Outlook:**

The outlook for Apple is mixed. While the company is taking steps to mitigate risks related to tariffs and is investing in AI, it faces challenges in the Chinese market and mixed sentiment regarding its AI capabilities. Dell appears to have a more positive outlook, with strong analyst recommendations and a focus on AI-driven products. HP Inc. faces challenges related to declining stock value and mixed analyst sentiment.


In [29]:
send_message('''What does the recent news say about Apple and the impact of tariffs? Over the past 2 month.''')

#### Natural language response

Between February 28, 2025, and April 28, 2025, news articles indicate that Apple is navigating a complex trade environment, particularly concerning tariffs.

Here's a summary of the key points:

*   **Tariff Impact:** Concerns loom over Apple's earnings report as analysts gauge the impact of tariffs on its business. Tariffs are expected to increase costs for businesses.
*   **China Dependence:** Apple's significant iPhone production in China exposes it to U.S. tariffs.
*   **Potential Shift to India:** Apple is reportedly planning to shift all production for U.S.-sold iPhones from China to India by the end of 2026, accelerating its supply chain diversification strategy amid trade war and tariff concerns.
*   **Trump's Stance:** The Trump administration has temporarily exempted smartphones, computers, and other electronic devices from tariffs on China imports.
*   **Analyst Views:** Wedbush's Dan Ives remains bullish on Apple in the long term, despite cutting Apple's price target.
*   **Political Pressure:** Senator Elizabeth Warren has criticized Tim Cook over alleged deals with the Trump administration to shield Apple from tariffs.
*   **Consumer Impact:** T-Mobile's CEO suggests that tariffs on phones will likely be absorbed by consumers.

# Conclusion

<span style="font-size:18px;">
For now that will have to do. Our Essy has a solid foundation but more could be done to organise metadata. No evaluation or validation has been performed (except fuzzing the prompt). Next steps include restructuring the vector database based on lessons learned. That'll be followed by plotting, multi-modal, and structured output. The last close date (generative) function can be temperamental. In the same way Gemini always feels regarding dates. I've learnt so much. I'm happy I decided to participate in the event! It really has been a joy to see Essy grow from random chat with Gemini into the foundation for a good-broker buddy. I hope you enjoy playing with this edition as much as I enjoyed building it!
</span>